# Retrieve Data from Database

In order to connect to the database, first, make sure:
1. …you have a local (running) database with the `hostname`, `database` name, `username`, and `port` number as specified in `/resources/config/gadb_pg_config.py`.
2. …the `airports` and `flights_and_weather` tables have data.

If the `airports` and `flights_and_weather` tables do not have data, you must first run `/download_and_prepare_datasets/02_prepare_and_store_data.ipynb` to populate the database.

In [1]:
import os
import time

In [2]:
os.chdir(os.path.join('..','resources'))

## Connect to Database

In [3]:
# Database configuration details
from config import gadb_pg_config as cfg

# To connect to SQL database
import sqlalchemy as db
# from sqlalchemy import create_engine, MetaData, Table, text, types

# To enter passwords without exposing them
from getpass import getpass

Most of the database information is already in `cfg` (above). However, you will have to enter your password below.

In [4]:
password = getpass('Enter database password')

Enter database password········


In [5]:
start_time = time.time()

In [6]:
db_string = f"postgresql+psycopg2://{cfg.username}:{password}@{cfg.hostname}:{cfg.port}/{cfg.database}"

In [7]:
engine = db.create_engine(
    future=True,
#     echo=True,
    url=db_string
)

## Access Database Tables and Create Table Aliases

In [8]:
db_meta = db.MetaData()

In [9]:
# Access the `airports` and `flights_and_weather` tables and assign them to variables
airports_table = db.Table('airports', db_meta, autoload_with=engine)
faw_table = db.Table('flights_and_weather', db_meta, autoload_with=engine)
faw = faw_table.alias('faw')
orig = airports_table.alias('orig')
dest = airports_table.alias('dest')

## Extract Full, Joined Dataset to Pandas Dataframe

In [10]:
from io import StringIO
import pandas as pd

In [11]:
# Function to extract from database query to Pandas dataframe
# Adapted from <https://towardsdatascience.com/optimizing-pandas-read-sql-for-postgres-f31cd7f707ab>
def read_sql_inmem(query, db_engine, **kwargs):
    copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
       query=query, head="HEADER"
    )
    conn = db_engine.raw_connection()
    cur = conn.cursor()
    store = StringIO()
    cur.copy_expert(copy_sql, store)
    store.seek(0)
    df = pd.read_csv(store, **kwargs)
    return df

In [12]:
# The query composed as an SQLAlchemy table object
get_all = (
    db.select([
        faw,
        orig.c.lat_decimal.label('origin_lat'),
        orig.c.lon_decimal.label('origin_lon'),
        dest.c.lat_decimal.label('destination_lat'),
        dest.c.lon_decimal.label('destination_lon')
    ])
    .select_from(
        faw
        .join(orig, orig.c.iata_code == faw.c.origin_airport, isouter=True)
        .join(dest, dest.c.iata_code == faw.c.destination_airport, isouter=True)
    )
    .order_by(faw.c.id)
)

In [13]:
def print_shape(df):
    df_rows, df_cols = df.shape
    print(f"{df_rows:,} rows × {df_cols:,} columns")

In [14]:
def df_details(df):
    return pd.concat(
        [
            df.dtypes,
            df.isna().sum(),
            pd.Series(
                data=[df[col].dropna().is_unique for col in df.columns],
                index=df.columns
            )
        ],
        axis=1,
        keys=['data_type','null_count','unique']
    )

In [15]:
# Run the query, and save the result to a dataframe
combined_df = read_sql_inmem(
    query=get_all,
#     query=get_all.limit(10).compile(engine, compile_kwargs={"literal_binds": True}),
    db_engine=engine,
    index_col='id',
    converters={'cancelled':(lambda x: True if x == 't' else False)}
)
# .sort_index(kind='mergesort')

print_shape(combined_df)

5,468,069 rows × 35 columns


In [16]:
with pd.option_context('display.max_columns',None):
    display(combined_df.head())

,carrier_code,flight_number,origin_airport,destination_airport,flight_date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,delay_carrier,delay_weather,delay_national_aviation_system,delay_security,delay_late_aircarft_arrival,cancelled,scheduled_departure_dt,scheduled_arrival_dt,actual_departure_dt,actual_arrival_dt,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,AS,121,SEA,ANC,2019-07-01,215,N611AS,0,-16,0,0,0,0,0,False,2019-07-01 00:40:00,2019-07-01 03:15:00,2019-07-01 00:40:00,2019-07-01 02:59:00,7.279302e+10,65.0,0.0,29.60,10.0,6.0,7.027253e+10,57.0,0.0,30.04,10.0,3.0,47.449,-122.309,61.174,-149.996
1,B6,1092,BQN,FLL,2019-07-01,155,N589JB,-19,-40,0,0,0,0,0,False,2019-07-01 01:26:00,2019-07-01 04:01:00,2019-07-01 01:07:00,2019-07-01 03:21:00,7.851401e+10,79.0,NaN,29.81,10.0,9.0,7.478301e+10,78.0,0.0,30.01,10.0,0.0,18.495,-67.129,26.072,-80.153
2,B6,668,PSE,MCO,2019-07-01,175,N662JB,73,69,0,0,0,0,69,False,2019-07-01 01:15:00,2019-07-01 04:10:00,2019-07-01 02:28:00,2019-07-01 05:19:00,NaN,NaN,NaN,NaN,NaN,NaN,7.220501e+10,76.0,0.0,29.93,10.0,3.0,18.008,-66.563,28.429,-81.316
3,F9,402,LAX,DEN,2019-07-01,143,N706FR,-1,-7,0,0,0,0,0,False,2019-07-01 00:59:00,2019-07-01 04:22:00,2019-07-01 00:58:00,2019-07-01 04:15:00,7.229502e+10,65.0,0.0,29.60,10.0,6.0,7.256500e+10,62.0,0.0,24.69,10.0,8.0,33.942,-118.408,39.858,-104.667
4,F9,790,PDX,DEN,2019-07-01,156,N350FR,21,10,0,0,0,0,0,False,2019-07-01 00:55:00,2019-07-01 04:31:00,2019-07-01 01:16:00,2019-07-01 04:41:00,7.269802e+10,66.0,0.0,29.92,10.0,6.0,7.256500e+10,62.0,0.0,24.70,8.0,10.0,45.589,-122.597,39.858,-104.667


In [17]:
df_details(combined_df)

,data_type,null_count,unique
carrier_code,object,0,False
flight_number,int64,0,False
origin_airport,object,0,False
destination_airport,object,0,False
flight_date,object,0,False
scheduled_elapsed_time,int64,0,False
tail_number,object,6973,False
departure_delay,int64,0,False
arrival_delay,int64,0,False
delay_carrier,int64,0,False


In [18]:
print(f'Elapsed time: {(time.time() - start_time):,.3f} seconds.')

Elapsed time: 382.805 seconds.


In [2]:
# from path import Path
import numpy as np
import pandas as pd

In [3]:
# Read in the data
ddir = r"/Users/neesha/Desktop/dataexport"
ddir
pt = os.path.join(ddir,'combined.csv')
pt
df = pd.read_csv(pt)
df.head()

,id,carrier_code,flight_number,origin_airport,destination_airport,flight_date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
0,0,AS,121,SEA,ANC,2019-07-01,215,N611AS,0,-16,...,7.027253e+10,57.0,0.0,30.04,10.0,3.0,47.449,-122.309,61.174,-149.996
1,1,B6,1092,BQN,FLL,2019-07-01,155,N589JB,-19,-40,...,7.478301e+10,78.0,0.0,30.01,10.0,0.0,18.495,-67.129,26.072,-80.153
2,2,B6,668,PSE,MCO,2019-07-01,175,N662JB,73,69,...,7.220501e+10,76.0,0.0,29.93,10.0,3.0,18.008,-66.563,28.429,-81.316
3,3,F9,402,LAX,DEN,2019-07-01,143,N706FR,-1,-7,...,7.256500e+10,62.0,0.0,24.69,10.0,8.0,33.942,-118.408,39.858,-104.667
4,4,F9,790,PDX,DEN,2019-07-01,156,N350FR,21,10,...,7.256500e+10,62.0,0.0,24.70,8.0,10.0,45.589,-122.597,39.858,-104.667


In [6]:
df.columns

Index(['id', 'carrier_code', 'flight_number', 'origin_airport',
       'destination_airport', 'flight_date', 'scheduled_elapsed_time',
       'tail_number', 'departure_delay', 'arrival_delay', 'delay_carrier',
       'delay_weather', 'delay_national_aviation_system', 'delay_security',
       'delay_late_aircarft_arrival', 'cancelled', 'scheduled_departure_dt',
       'scheduled_arrival_dt', 'actual_departure_dt', 'actual_arrival_dt',
       'station_x', 'hourlydrybulbtemperature_x', 'hourlyprecipitation_x',
       'hourlystationpressure_x', 'hourlyvisibility_x', 'hourlywindspeed_x',
       'station_y', 'hourlydrybulbtemperature_y', 'hourlyprecipitation_y',
       'hourlystationpressure_y', 'hourlyvisibility_y', 'hourlywindspeed_y',
       'origin_lat', 'origin_lon', 'destination_lat', 'destination_lon'],
      dtype='object')

In [7]:
df['cancelled'].value_counts()

f    5426150
t      41919
Name: cancelled, dtype: int64

In [10]:
df

,id,carrier_code,flight_number,origin_airport,destination_airport,flight_date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
0,0,AS,121,SEA,ANC,2019-07-01,215,N611AS,0,-16,...,7.027253e+10,57.0,0.0,30.04,10.0,3.0,47.449,-122.309,61.174,-149.996
1,1,B6,1092,BQN,FLL,2019-07-01,155,N589JB,-19,-40,...,7.478301e+10,78.0,0.0,30.01,10.0,0.0,18.495,-67.129,26.072,-80.153
2,2,B6,668,PSE,MCO,2019-07-01,175,N662JB,73,69,...,7.220501e+10,76.0,0.0,29.93,10.0,3.0,18.008,-66.563,28.429,-81.316
3,3,F9,402,LAX,DEN,2019-07-01,143,N706FR,-1,-7,...,7.256500e+10,62.0,0.0,24.69,10.0,8.0,33.942,-118.408,39.858,-104.667
4,4,F9,790,PDX,DEN,2019-07-01,156,N350FR,21,10,...,7.256500e+10,62.0,0.0,24.70,8.0,10.0,45.589,-122.597,39.858,-104.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468064,5512898,DL,3526,ATL,MGM,2019-05-31,59,N8923A,-1,-18,...,7.222601e+10,74.0,0.0,29.67,10.0,6.0,33.640,-84.427,NaN,NaN
5468065,5512899,DL,508,ATL,BHM,2019-05-31,55,N978DL,-3,-6,...,7.222801e+10,74.0,0.0,29.26,10.0,0.0,33.640,-84.427,33.563,-86.753
5468066,5512900,DL,1075,ATL,HSV,2019-05-31,57,N952AT,31,18,...,7.232300e+10,71.0,0.0,29.20,10.0,3.0,33.640,-84.427,NaN,NaN
5468067,5512901,DL,2436,ATL,BHM,2019-05-31,56,N916DL,28,13,...,7.222801e+10,67.0,0.0,29.24,10.0,0.0,33.640,-84.427,33.563,-86.753


In [11]:
df['scheduled_departure_dt']

0          2019-07-01 00:40:00
1          2019-07-01 01:26:00
2          2019-07-01 01:15:00
3          2019-07-01 00:59:00
4          2019-07-01 00:55:00
                  ...         
5468064    2019-05-31 21:06:00
5468065    2019-05-31 21:26:00
5468066    2019-05-31 22:44:00
5468067    2019-05-31 23:30:00
5468068    2019-05-31 23:38:00
Name: scheduled_departure_dt, Length: 5468069, dtype: object

In [12]:
df.columns

Index(['id', 'carrier_code', 'flight_number', 'origin_airport',
       'destination_airport', 'flight_date', 'scheduled_elapsed_time',
       'tail_number', 'departure_delay', 'arrival_delay', 'delay_carrier',
       'delay_weather', 'delay_national_aviation_system', 'delay_security',
       'delay_late_aircarft_arrival', 'cancelled', 'scheduled_departure_dt',
       'scheduled_arrival_dt', 'actual_departure_dt', 'actual_arrival_dt',
       'station_x', 'hourlydrybulbtemperature_x', 'hourlyprecipitation_x',
       'hourlystationpressure_x', 'hourlyvisibility_x', 'hourlywindspeed_x',
       'station_y', 'hourlydrybulbtemperature_y', 'hourlyprecipitation_y',
       'hourlystationpressure_y', 'hourlyvisibility_y', 'hourlywindspeed_y',
       'origin_lat', 'origin_lon', 'destination_lat', 'destination_lon'],
      dtype='object')

### 1.) origin_airport
### 2.) destination_airport
### 3.) departure_delay
### 4.) arrival_delay
### 5.) 'cancelled'
### 6.) station_x
### 7.) hourlydrybulbtemperature_x
### 8.) hourlyprecipitation_x
### 9.) hourlystationpressure_x
### 10.) 'hourlyvisibility_x', 
### 11.) 'hourlywindspeed_x',
### 12.) 'station_y', 
### 13.) 'hourlydrybulbtemperature_y', 
### 15.) 'hourlyprecipitation_y',
###   16.)  'hourlystationpressure_y',
### 17.) 'hourlyvisibility_y',
### 18.) 'hourlywindspeed_y',
### 19.) 'origin_lat', 
### 20 .) 'origin_lon', 
### 21.) 'destination_lat',
### 22.) 'destination_lon'

In [13]:
columns = ['origin_airport',
            'destination_airport',
            'departure_delay',
            'arrival_delay',
            'cancelled',
            'station_x',
            'hourlydrybulbtemperature_x',
            'hourlyprecipitation_x',
            'hourlystationpressure_x',
            'hourlyvisibility_x', 
            'hourlywindspeed_x',
            'station_y', 
            'hourlydrybulbtemperature_y', 
            'hourlyprecipitation_y',
            'hourlystationpressure_y',
            'hourlyvisibility_y',
            'hourlywindspeed_y',
            'origin_lat', 
            'origin_lon', 
            'destination_lat',
            'destination_lon']
columns

['origin_airport',
 'destination_airport',
 'departure_delay',
 'arrival_delay',
 'cancelled',
 'station_x',
 'hourlydrybulbtemperature_x',
 'hourlyprecipitation_x',
 'hourlystationpressure_x',
 'hourlyvisibility_x',
 'hourlywindspeed_x',
 'station_y',
 'hourlydrybulbtemperature_y',
 'hourlyprecipitation_y',
 'hourlystationpressure_y',
 'hourlyvisibility_y',
 'hourlywindspeed_y',
 'origin_lat',
 'origin_lon',
 'destination_lat',
 'destination_lon']

In [14]:
len(columns)

21

In [15]:
df = df[columns]

In [16]:
df

,origin_airport,destination_airport,departure_delay,arrival_delay,cancelled,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
0,SEA,ANC,0,-16,f,7.279302e+10,65.0,0.0,29.60,10.0,...,7.027253e+10,57.0,0.0,30.04,10.0,3.0,47.449,-122.309,61.174,-149.996
1,BQN,FLL,-19,-40,f,7.851401e+10,79.0,NaN,29.81,10.0,...,7.478301e+10,78.0,0.0,30.01,10.0,0.0,18.495,-67.129,26.072,-80.153
2,PSE,MCO,73,69,f,NaN,NaN,NaN,NaN,NaN,...,7.220501e+10,76.0,0.0,29.93,10.0,3.0,18.008,-66.563,28.429,-81.316
3,LAX,DEN,-1,-7,f,7.229502e+10,65.0,0.0,29.60,10.0,...,7.256500e+10,62.0,0.0,24.69,10.0,8.0,33.942,-118.408,39.858,-104.667
4,PDX,DEN,21,10,f,7.269802e+10,66.0,0.0,29.92,10.0,...,7.256500e+10,62.0,0.0,24.70,8.0,10.0,45.589,-122.597,39.858,-104.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468064,ATL,MGM,-1,-18,f,7.221901e+10,78.0,0.0,28.84,10.0,...,7.222601e+10,74.0,0.0,29.67,10.0,6.0,33.640,-84.427,NaN,NaN
5468065,ATL,BHM,-3,-6,f,7.221901e+10,77.0,0.0,28.84,10.0,...,7.222801e+10,74.0,0.0,29.26,10.0,0.0,33.640,-84.427,33.563,-86.753
5468066,ATL,HSV,31,18,f,7.221901e+10,74.0,0.0,28.84,10.0,...,7.232300e+10,71.0,0.0,29.20,10.0,3.0,33.640,-84.427,NaN,NaN
5468067,ATL,BHM,28,13,f,7.221901e+10,73.0,0.0,28.84,10.0,...,7.222801e+10,67.0,0.0,29.24,10.0,0.0,33.640,-84.427,33.563,-86.753


In [17]:
df['cancelled'].value_counts()

f    5426150
t      41919
Name: cancelled, dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5468069 entries, 0 to 5468068
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   origin_airport              object 
 1   destination_airport         object 
 2   departure_delay             int64  
 3   arrival_delay               int64  
 4   cancelled                   object 
 5   station_x                   float64
 6   hourlydrybulbtemperature_x  float64
 7   hourlyprecipitation_x       float64
 8   hourlystationpressure_x     float64
 9   hourlyvisibility_x          float64
 10  hourlywindspeed_x           float64
 11  station_y                   float64
 12  hourlydrybulbtemperature_y  float64
 13  hourlyprecipitation_y       float64
 14  hourlystationpressure_y     float64
 15  hourlyvisibility_y          float64
 16  hourlywindspeed_y           float64
 17  origin_lat                  float64
 18  origin_lon                  float64
 19  destination_lat      

In [19]:
df.isnull().sum()

origin_airport                     0
destination_airport                0
departure_delay                    0
arrival_delay                      0
cancelled                          0
station_x                       2073
hourlydrybulbtemperature_x      2073
hourlyprecipitation_x           9881
hourlystationpressure_x         2073
hourlyvisibility_x              2073
hourlywindspeed_x               2073
station_y                       2078
hourlydrybulbtemperature_y      2078
hourlyprecipitation_y           9896
hourlystationpressure_y         2078
hourlyvisibility_y              2078
hourlywindspeed_y               2078
origin_lat                    382438
origin_lon                    382438
destination_lat               382775
destination_lon               382775
dtype: int64

In [20]:
382438 / df.shape[0]

0.06994022935701799

In [21]:
df_new = df.dropna().copy()

In [22]:
df_new.isnull().sum()

origin_airport                0
destination_airport           0
departure_delay               0
arrival_delay                 0
cancelled                     0
station_x                     0
hourlydrybulbtemperature_x    0
hourlyprecipitation_x         0
hourlystationpressure_x       0
hourlyvisibility_x            0
hourlywindspeed_x             0
station_y                     0
hourlydrybulbtemperature_y    0
hourlyprecipitation_y         0
hourlystationpressure_y       0
hourlyvisibility_y            0
hourlywindspeed_y             0
origin_lat                    0
origin_lon                    0
destination_lat               0
destination_lon               0
dtype: int64

In [23]:
df_new['cancelled'].value_counts()

f    4674943
t      33957
Name: cancelled, dtype: int64

# Do Undersampling due to imbalance in data

In [24]:
df_new_f = df_new[df_new['cancelled'] == 'f'].sample(33957)
df_new_t = df_new[df_new['cancelled'] == 't']

In [25]:
df_new_f.shape

(33957, 21)

In [26]:
33957 * 2

67914

In [27]:
df_new_t.shape

(33957, 21)

In [28]:
df_final = pd.concat([df_new_f, df_new_t], axis = 0)# .reset_index(drop = True)

In [29]:
df_final.shape

(67914, 21)

In [30]:
df_final

,origin_airport,destination_airport,departure_delay,arrival_delay,cancelled,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
3531956,PHX,BFL,62,40,f,7.227802e+10,98.0,0.00,28.65,10.0,...,7.238402e+10,90.0,0.0,29.31,10.0,9.0,33.434,-112.008,35.433,-119.057
5337409,TPA,ATL,35,16,f,7.221101e+10,76.0,0.00,30.16,10.0,...,7.221901e+10,86.0,0.0,29.10,10.0,8.0,27.975,-82.533,33.640,-84.427
2488352,CAE,CLT,-3,-16,f,7.231001e+10,49.0,0.00,29.61,4.0,...,7.231401e+10,51.0,0.0,29.03,7.0,0.0,33.939,-81.119,35.214,-80.943
2881648,MDW,DEN,7,-6,f,7.253401e+10,71.0,0.00,29.36,10.0,...,7.256500e+10,71.0,0.0,24.65,10.0,6.0,41.786,-87.752,39.858,-104.667
3913784,SEA,SLC,-3,-16,f,7.279302e+10,68.0,0.00,29.52,10.0,...,7.257202e+10,84.0,0.0,25.69,10.0,9.0,47.449,-122.309,40.788,-111.978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463047,CAE,CLT,0,0,t,7.231001e+10,86.0,0.00,29.56,10.0,...,7.231401e+10,71.0,0.0,29.05,10.0,0.0,33.939,-81.119,35.214,-80.943
5463530,RDU,EWR,0,0,t,7.230601e+10,70.0,0.04,29.30,10.0,...,7.250201e+10,75.0,0.0,29.77,10.0,9.0,35.877,-78.787,40.692,-74.169
5464231,CAE,DCA,0,0,t,7.231001e+10,86.0,0.00,29.56,10.0,...,7.240501e+10,77.0,0.0,29.76,10.0,6.0,33.939,-81.119,38.852,-77.037
5464827,TYS,CLT,0,0,t,7.232601e+10,72.0,0.00,28.88,10.0,...,7.231401e+10,70.0,0.0,29.08,10.0,3.0,35.812,-83.993,35.214,-80.943


In [31]:
df_final = df_final.sample(frac=1).reset_index(drop = True)

In [32]:
df_final

,origin_airport,destination_airport,departure_delay,arrival_delay,cancelled,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
0,DFW,MSY,-7,-17,f,7.225900e+10,60.0,0.00,29.25,10.0,...,7.223101e+10,64.0,0.0,30.00,10.0,3.0,32.896,-97.037,29.993,-90.258
1,MCI,BNA,-2,-14,f,7.244600e+10,72.0,0.00,29.00,10.0,...,7.232701e+10,89.0,0.0,29.49,10.0,8.0,39.297,-94.714,36.124,-86.678
2,BOS,MKE,0,0,t,7.250901e+10,73.0,0.00,29.79,10.0,...,7.264001e+10,67.0,0.0,29.34,10.0,0.0,42.364,-71.005,42.947,-87.896
3,LAS,SNA,-3,-8,f,7.238602e+10,71.0,0.00,27.78,10.0,...,7.229779e+10,70.0,0.0,29.99,8.0,5.0,36.080,-115.152,33.676,-117.868
4,CLT,TLH,-3,-3,f,7.231401e+10,43.0,0.00,29.46,10.0,...,7.221409e+10,61.0,0.0,30.08,10.0,5.0,35.214,-80.943,30.396,-84.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67909,SAN,SEA,1,1,f,7.229002e+10,63.0,0.00,29.93,10.0,...,7.279302e+10,57.0,0.0,29.53,10.0,5.0,32.733,-117.189,47.449,-122.309
67910,COS,DEN,-2,-5,f,7.246609e+10,44.0,0.00,23.94,10.0,...,7.256500e+10,46.0,0.0,24.55,10.0,0.0,38.806,-104.700,39.858,-104.667
67911,SUX,ORD,2,17,f,7.255701e+10,44.0,0.00,28.82,10.0,...,7.253009e+10,51.0,0.0,29.41,10.0,14.0,42.402,-96.384,41.979,-87.904
67912,ORD,BDL,-1,0,t,7.253009e+10,73.0,0.42,29.19,8.0,...,7.250801e+10,75.0,0.0,29.75,10.0,10.0,41.979,-87.904,41.939,-72.683


In [33]:
df_final['cancelled'].value_counts()

f    33957
t    33957
Name: cancelled, dtype: int64

In [34]:
df_final.sample()

,origin_airport,destination_airport,departure_delay,arrival_delay,cancelled,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,...,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
17409,AUS,STL,0,0,t,7.225401e+10,81.0,0.0,29.18,10.0,...,7.243401e+10,27.0,0.01,29.23,5.0,14.0,30.194,-97.67,38.748,-90.36


In [35]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67914 entries, 0 to 67913
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   origin_airport              67914 non-null  object 
 1   destination_airport         67914 non-null  object 
 2   departure_delay             67914 non-null  int64  
 3   arrival_delay               67914 non-null  int64  
 4   cancelled                   67914 non-null  object 
 5   station_x                   67914 non-null  float64
 6   hourlydrybulbtemperature_x  67914 non-null  float64
 7   hourlyprecipitation_x       67914 non-null  float64
 8   hourlystationpressure_x     67914 non-null  float64
 9   hourlyvisibility_x          67914 non-null  float64
 10  hourlywindspeed_x           67914 non-null  float64
 11  station_y                   67914 non-null  float64
 12  hourlydrybulbtemperature_y  67914 non-null  float64
 13  hourlyprecipitation_y       679

In [36]:
df_final['origin_airport'].value_counts()

DFW    4606
ORD    3746
DEN    2642
ATL    2235
EWR    2209
       ... 
BFM       4
SPN       2
GUM       2
CDC       2
AKN       1
Name: origin_airport, Length: 203, dtype: int64

In [37]:
4603*0.04

184.12

In [38]:
main_airport_list = df_final['origin_airport'].value_counts()[df_final['origin_airport'].value_counts() > 4603*0.04].index
main_airport_list

Index(['DFW', 'ORD', 'DEN', 'ATL', 'EWR', 'CLT', 'IAH', 'LGA', 'MCO', 'PHL',
       'LAX', 'DCA', 'BOS', 'SFO', 'DTW', 'MDW', 'LAS', 'PHX', 'DAL', 'BWI',
       'SEA', 'MSP', 'JFK', 'FLL', 'MIA', 'MSY', 'BNA', 'IAD', 'HOU', 'STL',
       'SAN', 'AUS', 'SLC', 'RDU', 'TPA', 'MCI', 'CLE', 'PDX', 'PIT', 'CMH',
       'IND', 'JAX', 'SAT', 'CHS', 'SJC', 'CVG', 'ORF', 'SMF', 'SNA', 'OAK',
       'MKE', 'OKC', 'BDL', 'MEM', 'RIC', 'PBI', 'BUF', 'HNL', 'OMA', 'BHM',
       'BUR', 'SAV', 'ABQ', 'SJU', 'SYR', 'GRR', 'ALB', 'ROC', 'TUL', 'MYR',
       'RSW', 'TYS', 'PVD', 'ELP', 'HPN', 'ANC'],
      dtype='object')

In [39]:
len(main_airport_list)

76

In [40]:
df_final['origin_airport'] = df_final['origin_airport'].apply(lambda x : x if x in main_airport_list else 'other')

In [41]:
df_final['origin_airport'].value_counts()

other    6978
DFW      4606
ORD      3746
DEN      2642
ATL      2235
         ... 
TYS       198
PVD       197
ELP       193
HPN       192
ANC       189
Name: origin_airport, Length: 77, dtype: int64

In [42]:
df_final['destination_airport'].value_counts()

DFW    4502
ORD    3825
DEN    2637
ATL    2400
EWR    2230
       ... 
BFM       3
GUM       2
SPN       2
PPG       1
IAG       1
Name: destination_airport, Length: 203, dtype: int64

In [44]:
4610 * 0.04

184.4

In [45]:
main_destination_airport_list = df_final['destination_airport'].value_counts()[df_final['destination_airport'].value_counts() > 4610 * 0.04].index
main_destination_airport_list

Index(['DFW', 'ORD', 'DEN', 'ATL', 'EWR', 'CLT', 'IAH', 'LGA', 'MCO', 'PHL',
       'LAX', 'DCA', 'BOS', 'SFO', 'DTW', 'PHX', 'MDW', 'LAS', 'BWI', 'SEA',
       'MSP', 'JFK', 'FLL', 'DAL', 'MIA', 'IAD', 'HOU', 'BNA', 'SAN', 'SLC',
       'RDU', 'STL', 'MSY', 'AUS', 'MCI', 'TPA', 'CLE', 'PDX', 'CMH', 'PIT',
       'IND', 'CVG', 'SJC', 'CHS', 'JAX', 'SAT', 'MKE', 'SMF', 'ORF', 'OKC',
       'SNA', 'HNL', 'BUF', 'OAK', 'PBI', 'MEM', 'RIC', 'SAV', 'OMA', 'BDL',
       'GRR', 'BUR', 'ALB', 'ABQ', 'BHM', 'SYR', 'TUL', 'SJU', 'ANC', 'ROC',
       'PVD'],
      dtype='object')

In [46]:
len(main_destination_airport_list)

71

In [47]:
df_final['destination_airport'] = df_final['destination_airport'].apply(lambda x : x if x in main_destination_airport_list else 'other')

In [48]:
df_final['destination_airport'].value_counts()

other    7734
DFW      4502
ORD      3825
DEN      2637
ATL      2400
         ... 
TUL       222
SJU       217
ANC       208
ROC       200
PVD       197
Name: destination_airport, Length: 72, dtype: int64

In [49]:
# Separate the features (X) from the target (y)
y = df_final["cancelled"]
X = df_final.drop(columns="cancelled")

In [50]:
X

,origin_airport,destination_airport,departure_delay,arrival_delay,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
0,DFW,MSY,-7,-17,7.225900e+10,60.0,0.00,29.25,10.0,7.0,7.223101e+10,64.0,0.0,30.00,10.0,3.0,32.896,-97.037,29.993,-90.258
1,MCI,BNA,-2,-14,7.244600e+10,72.0,0.00,29.00,10.0,9.0,7.232701e+10,89.0,0.0,29.49,10.0,8.0,39.297,-94.714,36.124,-86.678
2,BOS,MKE,0,0,7.250901e+10,73.0,0.00,29.79,10.0,14.0,7.264001e+10,67.0,0.0,29.34,10.0,0.0,42.364,-71.005,42.947,-87.896
3,LAS,SNA,-3,-8,7.238602e+10,71.0,0.00,27.78,10.0,7.0,7.229779e+10,70.0,0.0,29.99,8.0,5.0,36.080,-115.152,33.676,-117.868
4,CLT,other,-3,-3,7.231401e+10,43.0,0.00,29.46,10.0,13.0,7.221409e+10,61.0,0.0,30.08,10.0,5.0,35.214,-80.943,30.396,-84.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67909,SAN,SEA,1,1,7.229002e+10,63.0,0.00,29.93,10.0,5.0,7.279302e+10,57.0,0.0,29.53,10.0,5.0,32.733,-117.189,47.449,-122.309
67910,other,DEN,-2,-5,7.246609e+10,44.0,0.00,23.94,10.0,5.0,7.256500e+10,46.0,0.0,24.55,10.0,0.0,38.806,-104.700,39.858,-104.667
67911,other,ORD,2,17,7.255701e+10,44.0,0.00,28.82,10.0,9.0,7.253009e+10,51.0,0.0,29.41,10.0,14.0,42.402,-96.384,41.979,-87.904
67912,ORD,BDL,-1,0,7.253009e+10,73.0,0.42,29.19,8.0,13.0,7.250801e+10,75.0,0.0,29.75,10.0,10.0,41.979,-87.904,41.939,-72.683


In [51]:
# Use the train_test_split function to create training and testing subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y, test_size=0.1)
X_train.shape

(61122, 20)

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((61122, 20), (6792, 20), (61122,), (6792,))

In [53]:
X_train

,origin_airport,destination_airport,departure_delay,arrival_delay,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
22682,PHL,IAH,-6,-4,7.240801e+10,73.0,0.00,30.14,10.0,14.0,7.224301e+10,78.0,0.00,29.96,10.0,14.0,39.872,-75.241,29.980,-95.340
47726,JFK,RDU,0,0,7.448609e+10,83.0,0.00,29.83,10.0,7.0,7.230601e+10,93.0,0.00,29.44,10.0,8.0,40.640,-73.779,35.877,-78.787
14513,BWI,JAX,0,0,7.240609e+10,95.0,0.00,29.71,10.0,13.0,7.220601e+10,79.0,0.03,29.62,10.0,20.0,39.175,-76.668,30.494,-81.688
11411,TPA,ATL,-6,-21,7.221101e+10,87.0,0.00,29.99,10.0,3.0,7.221901e+10,78.0,0.00,28.93,10.0,7.0,27.975,-82.533,33.640,-84.427
3763,BNA,JFK,-8,-13,7.232701e+10,63.0,0.00,29.49,10.0,5.0,7.448609e+10,77.0,0.00,30.03,10.0,14.0,36.124,-86.678,40.640,-73.779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10635,TPA,IAH,-7,-12,7.221101e+10,83.0,0.00,29.88,10.0,8.0,7.224301e+10,69.0,0.00,29.77,10.0,6.0,27.975,-82.533,29.980,-95.340
14355,STL,CMH,228,221,7.243401e+10,63.0,0.65,28.98,9.0,3.0,7.242801e+10,56.0,0.00,29.22,10.0,8.0,38.748,-90.360,39.998,-82.892
63476,DEN,other,0,0,7.256500e+10,59.0,0.00,24.60,10.0,13.0,7.234031e+10,82.0,0.26,29.49,10.0,6.0,39.858,-104.667,34.729,-92.224
53284,MSP,MCO,0,0,7.265801e+10,71.0,0.00,29.24,10.0,3.0,7.220501e+10,84.0,0.00,29.68,10.0,11.0,44.880,-93.217,28.429,-81.316


In [54]:
X_train['station_x'].max(), X_train['station_x'].min()

(91285021504.0, 70026027502.0)

In [55]:
y_test

12060    t
28417    f
18878    f
46212    t
10836    t
        ..
38880    f
64097    f
23128    f
15989    f
58297    f
Name: cancelled, Length: 6792, dtype: object

In [56]:
from sklearn.compose import ColumnTransformer

In [57]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [58]:
Le = LabelEncoder()

In [59]:
y_train[:5]

22682    f
47726    t
14513    t
11411    f
3763     f
Name: cancelled, dtype: object

In [60]:
y_test[:5]

12060    t
28417    f
18878    f
46212    t
10836    t
Name: cancelled, dtype: object

In [61]:
y_train_cln = Le.fit_transform(y_train)

In [62]:
y_train_cln

array([0, 1, 1, ..., 1, 1, 0])

In [63]:
Le.classes_

array(['f', 't'], dtype=object)

In [64]:
y_test_cln = Le.transform(y_test)

In [65]:
y_test_cln

array([1, 0, 0, ..., 0, 0, 0])

In [66]:
X_train.dtypes

origin_airport                 object
destination_airport            object
departure_delay                 int64
arrival_delay                   int64
station_x                     float64
hourlydrybulbtemperature_x    float64
hourlyprecipitation_x         float64
hourlystationpressure_x       float64
hourlyvisibility_x            float64
hourlywindspeed_x             float64
station_y                     float64
hourlydrybulbtemperature_y    float64
hourlyprecipitation_y         float64
hourlystationpressure_y       float64
hourlyvisibility_y            float64
hourlywindspeed_y             float64
origin_lat                    float64
origin_lon                    float64
destination_lat               float64
destination_lon               float64
dtype: object

In [67]:
obj_col = X_train.dtypes[X.dtypes == 'object'].index.values
num_col = X_train.dtypes[X.dtypes != 'object'].index.values

In [68]:
len(obj_col), len(num_col)

(2, 18)

In [69]:
obj_col

array(['origin_airport', 'destination_airport'], dtype=object)

In [70]:
num_col

array(['departure_delay', 'arrival_delay', 'station_x',
       'hourlydrybulbtemperature_x', 'hourlyprecipitation_x',
       'hourlystationpressure_x', 'hourlyvisibility_x',
       'hourlywindspeed_x', 'station_y', 'hourlydrybulbtemperature_y',
       'hourlyprecipitation_y', 'hourlystationpressure_y',
       'hourlyvisibility_y', 'hourlywindspeed_y', 'origin_lat',
       'origin_lon', 'destination_lat', 'destination_lon'], dtype=object)

In [71]:
X.shape

(67914, 20)

In [75]:
X_train

,origin_airport,destination_airport,departure_delay,arrival_delay,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
22682,PHL,IAH,-6,-4,7.240801e+10,73.0,0.00,30.14,10.0,14.0,7.224301e+10,78.0,0.00,29.96,10.0,14.0,39.872,-75.241,29.980,-95.340
47726,JFK,RDU,0,0,7.448609e+10,83.0,0.00,29.83,10.0,7.0,7.230601e+10,93.0,0.00,29.44,10.0,8.0,40.640,-73.779,35.877,-78.787
14513,BWI,JAX,0,0,7.240609e+10,95.0,0.00,29.71,10.0,13.0,7.220601e+10,79.0,0.03,29.62,10.0,20.0,39.175,-76.668,30.494,-81.688
11411,TPA,ATL,-6,-21,7.221101e+10,87.0,0.00,29.99,10.0,3.0,7.221901e+10,78.0,0.00,28.93,10.0,7.0,27.975,-82.533,33.640,-84.427
3763,BNA,JFK,-8,-13,7.232701e+10,63.0,0.00,29.49,10.0,5.0,7.448609e+10,77.0,0.00,30.03,10.0,14.0,36.124,-86.678,40.640,-73.779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10635,TPA,IAH,-7,-12,7.221101e+10,83.0,0.00,29.88,10.0,8.0,7.224301e+10,69.0,0.00,29.77,10.0,6.0,27.975,-82.533,29.980,-95.340
14355,STL,CMH,228,221,7.243401e+10,63.0,0.65,28.98,9.0,3.0,7.242801e+10,56.0,0.00,29.22,10.0,8.0,38.748,-90.360,39.998,-82.892
63476,DEN,other,0,0,7.256500e+10,59.0,0.00,24.60,10.0,13.0,7.234031e+10,82.0,0.26,29.49,10.0,6.0,39.858,-104.667,34.729,-92.224
53284,MSP,MCO,0,0,7.265801e+10,71.0,0.00,29.24,10.0,3.0,7.220501e+10,84.0,0.00,29.68,10.0,11.0,44.880,-93.217,28.429,-81.316


In [76]:
# # 77
# a, 
# a, 
# b, 
# c, 
# b, 
# c, 
# a => 3 0 0 0
# 1st(a) 2nd(b) 3rd(c)
# 1        0      0
# 1        0      0
# 0         1     0
# 0        0      1


In [77]:
clf = ColumnTransformer([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown = 'ignore'), obj_col),
    ('ss', StandardScaler(), num_col)
], remainder='passthrough')

In [189]:
# clf

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 array(['origin_airport', 'destination_airport'], dtype=object)),
                                ('ss', StandardScaler(),
                                 array(['departure_delay', 'arrival_delay', 'station_x',
       'hourlydrybulbtemperature_x', 'hourlyprecipitation_x',
       'hourlystationpressure_x', 'hourlyvisibility_x',
       'hourlywindspeed_x', 'station_y', 'hourlydrybulbtemperature_y',
       'hourlyprecipitation_y', 'hourlystationpressure_y',
       'hourlyvisibility_y', 'hourlywindspeed_y', 'origin_lat',
       'origin_lon', 'destination_lat', 'destination_lon'], dtype=object))])

In [78]:
X_train

,origin_airport,destination_airport,departure_delay,arrival_delay,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
22682,PHL,IAH,-6,-4,7.240801e+10,73.0,0.00,30.14,10.0,14.0,7.224301e+10,78.0,0.00,29.96,10.0,14.0,39.872,-75.241,29.980,-95.340
47726,JFK,RDU,0,0,7.448609e+10,83.0,0.00,29.83,10.0,7.0,7.230601e+10,93.0,0.00,29.44,10.0,8.0,40.640,-73.779,35.877,-78.787
14513,BWI,JAX,0,0,7.240609e+10,95.0,0.00,29.71,10.0,13.0,7.220601e+10,79.0,0.03,29.62,10.0,20.0,39.175,-76.668,30.494,-81.688
11411,TPA,ATL,-6,-21,7.221101e+10,87.0,0.00,29.99,10.0,3.0,7.221901e+10,78.0,0.00,28.93,10.0,7.0,27.975,-82.533,33.640,-84.427
3763,BNA,JFK,-8,-13,7.232701e+10,63.0,0.00,29.49,10.0,5.0,7.448609e+10,77.0,0.00,30.03,10.0,14.0,36.124,-86.678,40.640,-73.779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10635,TPA,IAH,-7,-12,7.221101e+10,83.0,0.00,29.88,10.0,8.0,7.224301e+10,69.0,0.00,29.77,10.0,6.0,27.975,-82.533,29.980,-95.340
14355,STL,CMH,228,221,7.243401e+10,63.0,0.65,28.98,9.0,3.0,7.242801e+10,56.0,0.00,29.22,10.0,8.0,38.748,-90.360,39.998,-82.892
63476,DEN,other,0,0,7.256500e+10,59.0,0.00,24.60,10.0,13.0,7.234031e+10,82.0,0.26,29.49,10.0,6.0,39.858,-104.667,34.729,-92.224
53284,MSP,MCO,0,0,7.265801e+10,71.0,0.00,29.24,10.0,3.0,7.220501e+10,84.0,0.00,29.68,10.0,11.0,44.880,-93.217,28.429,-81.316


In [79]:
X_train_cln = clf.fit_transform(X_train)
X_test_cln = clf.transform(X_test)

In [211]:
X_train_cln

array([[ 0.        ,  0.        ,  0.        , ..., -0.06998344,
         0.02740703,  0.93861357],
       [ 0.        ,  0.        ,  0.        , ..., -0.33411739,
        -0.68428494, -0.28461912],
       [ 0.        ,  0.        ,  0.        , ...,  0.24516222,
        -1.1861799 ,  0.10596711],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.88974868,
         1.08026095,  0.38643812],
       [ 0.        ,  0.        ,  0.        , ..., -0.29657039,
        -0.84343523, -0.60151878],
       [ 0.        ,  0.        ,  0.        , ..., -1.79560316,
        -0.53849975, -1.52947206]])

In [80]:
# X_train => X_train_cln
# X_train => X_tes_cln

In [81]:
X_train_cln.shape, X_test_cln.shape

((61122, 167), (6792, 167))

In [82]:
# Create Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [84]:
# Fit(Train) the data
classifier.fit(X_train_cln, y_train_cln)

/Users/neesha/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [85]:
# Make predictions using the test data
y_pred = classifier.predict(X_test_cln)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Cancelled": y_test_cln
}).reset_index(drop=True)
results.head()

,Prediction,Cancelled
0,0,1
1,0,0
2,0,0
3,1,1
4,0,1


In [86]:
# Validate using test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test_cln, y_pred)

0.7196702002355713

In [87]:
# Generate confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_cln, y_pred)

array([[2428,  968],
       [ 936, 2460]])

In [89]:
# Generate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_cln, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.71      0.72      3396
           1       0.72      0.72      0.72      3396

    accuracy                           0.72      6792
   macro avg       0.72      0.72      0.72      6792
weighted avg       0.72      0.72      0.72      6792

